<a href="https://colab.research.google.com/github/donghwi2022/ds-section1-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 다음 분기에 어떤 게임을 설계해야 할까

# **Part 1. 필요 라이브러리 및 데이터 로드**
- 필요 라이브러리 import 및 한글 깨짐문제 해결
- csv 파일 load

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#한글 깨짐문제 해결
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumBarunGothic');

In [ ]:
# url로 필요 csv 파일 불러오기
url = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv"
df = pd.read_csv(url)
df.head()

# **Part 2. 데이터 전처리 진행**

In [ ]:
df.info()

In [ ]:
# 첫번째 column은 index를 나타내므로 해당 열 제거
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head()

In [ ]:
# 결측치 확인
df.isnull().sum()

In [ ]:
# 데이터 양이 충분하므로, 결측치가 존재하는 데이터 삭제
df.dropna(inplace = True)
df.info()

In [ ]:
# 중복값 확인(게임 명, 지원되는 플랫폼, 출시 연도, 장르, 게임회사 명이 같은 row는 중복데이터로 간주)
df[df.duplicated(['Name', 'Platform', 'Year', 'Genre', 'Publisher'])]

In [ ]:
# 중복되는 데이터 확인
df[(df.Name == 'Madden NFL 13') & (df.Platform == 'PS3') & (df.Year == 2012.0) & (df.Genre == 'Sports') & (df.Publisher == 'Electronic Arts')]

In [ ]:
# 중복되는 데이터 중 정확한 데이터를 알 수 없기 때문에 해당 row를 전부 삭제
duplicate_index = df[df.duplicated(['Name', 'Platform', 'Year', 'Genre', 'Publisher'])].index
duplicated_all_index = []
for i in range(len(duplicate_index)) :
    duplicated_data = df.loc[duplicate_index[i]].Name, df.loc[duplicate_index[i]].Platform, df.loc[duplicate_index[i]].Year, df.loc[duplicate_index[i]].Genre, df.loc[duplicate_index[i]].Publisher
    duplicated_all_index.extend(df[(df.Name == duplicated_data[0]) & (df.Platform == duplicated_data[1]) & (df.Year == duplicated_data[2]) & (df.Genre == duplicated_data[3]) & (df.Publisher == duplicated_data[4])].index)
    # append를 사용할 경우 배열에 배열을 추가할 시, 배열이 그대로 유지되지만 extend는 쪼개서 한 개씩 들어감
df.drop(duplicated_all_index, inplace = True)
df.info()

In [ ]:
# 출고량에서 숫자가 아닌 데이터 출력
for i in range(5, 9) : 
    df_1 = df.iloc[:, i].str.extract(r'([^\.\d])')
    print(df.columns[i], df_1[0].unique())


In [ ]:
# 출고량 단위 맞춤
cols = df.columns[5:9] # 출고량에 해당하는 column 명 저장

# K 단위를 M 단위로 맞추기 위함 함수 작성
def k_to_m(x) : 
    return x/1000 

for i in cols : 
    df[i] = df[i].str.replace('M', '') # M은 문자만 삭제
    df1 = df[df.loc[:, i].str.contains('K')].loc[:, i].str.replace('K', '').astype(int).apply(k_to_m) # K가 들어가는 데이터를 단위에 맞춰서 변형한 후 df1에 저장
    for j in df1.index : 
        df.loc[j, i] = df1.loc[j] # df1의 값을 바탕으로 df의 값을 변경
    df[i] = df[i].astype(float) # df의 타입 변경
df.head(11)

In [ ]:
# 연도도 정수형태로 변경 후, 연도의 데이터를 확인
df['Year'] = df['Year'].astype(int)
df.Year.unique()

In [ ]:
# 연도의 형식에 맞지 않는 데이터 변경
def year(x) :
    if x < 22 :
        return x + 2000
    elif (x >= 22) & (x < 100) :
        return x + 1900
    else :
        return x
df['Year'] = df['Year'].apply(year)
df.Year.unique()

In [ ]:
# 게임의 출고량 총합을 나타내는 column 작성
df['Sales_Total'] = df['EU_Sales'] + df['JP_Sales'] + df['NA_Sales'] + df['Other_Sales']

In [ ]:
# 최종 데이터 형태 확인
df.info()

In [ ]:
df.head()

# **Part 3. 다음 분기에 설계할 게임에 대한 기획 진행**
## 게임 회사는 'Idea Factory'
## 지금이 2016년도라 생각하고 2017년도에 설계할 게임에 대한 기획 진행
   (2017년부터는 데이터가 거의 없기 때문에)

In [ ]:
# df = 전체 데이터, df_co = 내가 정한 회사의 데이터
# 다음 분기를 계획하는 것이기 때문에 최근 10년의 데이터를 바탕으로 기획 진행
df_all = df[(df.Year >= 2012) & (df.Year < 2017)]
df_co = df[(df.Year >= 2012) & (df.Year < 2017) & (df.Publisher == 'Idea Factory')]
df_co.info()

In [ ]:
df_co.head()

In [ ]:
df_all.info()

In [ ]:
df_all.head()

## **Question 1. 지역에 따라 선호하는 게임 장르가 다를까**

In [ ]:
# 장르로 그룹화
def genre_by_region(df) :
    df_by_Genre = df.groupby(['Genre'])[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].sum() # 장르로 그룹화
    df_by_region = df_by_Genre.transpose() # 지역별로 확인하기 전치행렬

    #시각화
    df_by_region.plot.bar(figsize=(10,6))
    plt.xlabel('지역', size = 20)
    plt.xticks(rotation = 0, size = 15)
    plt.ylabel('판매량\n(단위 : 백만)', size = 15, rotation = 0, labelpad = 40)
    plt.suptitle('지역별 장르의 판매량', size = 25)
    if len(df) <= 100 : 
        plt.title('by Idea Factory', size = 15)
    else : 
        plt.title('by 모든 게임회사', size = 15)
    plt.show()

In [ ]:
genre_by_region(df_co)

In [ ]:
genre_by_region(df_all)

## **Question 2. 연도별 게임의 트렌드가 있을까**

In [ ]:
df1 = df_all.groupby(['Genre', 'Year'], as_index=False)[['Sales_Total']].sum() # 장르로 그룹화
df1.head()

In [ ]:
df1.Genre.unique()

In [ ]:
genre_list = df1.Genre.unique()
plt.figure(figsize = (10, 6))
for i in range(len(genre_list)) :
    plt.plot(df1[df1.Genre == genre_list[i]]['Year'], df1[df1.Genre == genre_list[i]]['Sales_Total'], label = genre_list[i])
    plt.xlabel('연도', size = 20)
    plt.ylabel('판매량\n(단위 : 백만)', size = 15, rotation = 0, labelpad = 40)
    plt.xticks(range(2012, 2017, 1))
    plt.legend()

In [ ]:
df2 = df_all.groupby(['Platform', 'Year'], as_index=False)[['Sales_Total']].sum() # 장르로 그룹화
df2.head()

In [ ]:
df2.Platform.unique()

In [ ]:
genre_list = df2.Platform.unique()
plt.figure(figsize = (10, 6))
for i in range(len(genre_list)) :
    plt.plot(df2[df2.Platform == genre_list[i]]['Year'], df2[df2.Platform == genre_list[i]]['Sales_Total'], label = genre_list[i])
    plt.xlabel('연도', size = 20)
    plt.ylabel('판매량\n(단위 : 백만)', size = 15, rotation = 0, labelpad = 40)
    plt.legend()

In [ ]:
df_all[df_all.Year >= 2017]

## **Question 3. 출고량이 높은 게임에 대한 분석 및 시각화**